<a href="https://colab.research.google.com/github/ahmededrisahmed/fmcs39130/blob/main/UpdateLSTMCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First we will start with the imports:**

In [ ]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import io
import re
from nltk.stem import PorterStemmer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [ ]:
from google.colab import files 
uploaded = files.upload()

In [ ]:
# data = pd.read_csv('amazon_alexa.tsv')

In [ ]:
# Now use the below code to read the csv in pandas' dataframe
data = pd.read_csv(io.StringIO(uploaded['amazon_alexa.tsv'].decode('utf-8')), sep='\t')
data = data[data.verified_reviews != "Neutral"]
data = data[['verified_reviews', 'feedback']]
# if(data['feedback'] == 1).any():
#   print(data['verified_reviews'])

In [ ]:
def cleanData(fiald, feedback):
    data[fiald] = data[fiald].apply(lambda x: x.lower())
    data[fiald] = data[fiald].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

    print(data[ data[feedback] == 1 ].size)
    print(data[ data[feedback] == 0 ].size)

    for idx,row in data.iterrows():
      row[0] = row[0].replace('rt',' ')
      
cleanData('verified_reviews', 'feedback')


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['verified_reviews'].values)
X = tokenizer.texts_to_sequences(data['verified_reviews'].values)
X = pad_sequences(X)
print(X)

In [ ]:
Y = pd.get_dummies(data['feedback']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)
print(Y_test)
print(X_test)

In [ ]:
embed_dim = 128
lstm_out = 196
max_fatures = 2000
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32
history = model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

In [ ]:
# batch_size = 32
# model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 
hist_df

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# y_pred = model.predict(X_test)
# # print(y_pred)
# s = []
# for i in range(y_pred):
#   if(y_pred[i] >= 0.5).all():
#     s.append(1)
#   else:
#     s.append(0)

# print(s)

In [ ]:
from google.colab import files 
uploaded = files.upload()

In [ ]:
# Now use the below code to read the csv in pandas' dataframe
df_text = pd.read_csv('twiiter_api_hash_tag.csv')
df_text.head()

In [ ]:
df_text.value_counts()

In [ ]:
df_text['Tweet'] = df_text['Tweet'].apply(lambda x: x.lower())
df_text['Tweet'] = df_text['Tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# for idx,row in df_text.iterrows():
#   row[0] = row[0].replace('rt','')

In [ ]:
df_text['Tweet']

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_text['Tweet'].values)
test_data = tokenizer.texts_to_sequences(df_text['Tweet'].values)
print(test_data)

In [ ]:
# history = model.fit(test_data, epochs = 1, batch_size=128, verbose = 2)

pad_test = pad_sequences(test_data, padding="post", maxlen=32)
pad_test

In [ ]:
mod_pridect = model.predict(pad_test)
mod_pridect

In [ ]:
pl = []
for i in range(len(mod_pridect)):
  for j in mod_pridect[i]:
    if (j >= 0.5).all():
      pl.append(1)
    else:
      pl.append(0)

# print(pl)

s = []
for i in range(len(pl)):
  if pl[i] >=0.5:
    s.append('positive')
  else:
    s.append('negative')
print(s)

In [ ]:
# sp = PorterStemmer()
# data_new = df_text['Tweet']
# data_array = []
# data_new
# for i in range(len(data_new)):
#   data_array.append(data_new)

# print(data_array)

In [ ]:
t  = ['love', 'love', 'love', 'love', 'love', 'love']
tk = tokenizer.texts_to_sequences(t)
# print(tk)
MAX_LEN = len(tk)
pss = pad_sequences(tk, padding = "post", maxlen = 32)
# print(pss)
p = model.predict(pss)
print(p)
pl = []
for i in p:
  if (i >= 0.5).all():
    pl.append(1)
  else:
    pl.append(0)
for i in range(len(t)):
  if pl == 1:
    s = 'positive'
  else:
    s = 'negative'
print(s)


In [ ]:
# pip install -q transformers

In [ ]:
# from transformers import pipeline

In [ ]:
# # f = df_text['Tweet'][0]
# data_new = df_text['Tweet']
# data_array = []
# data_new
# for i in range(len(data_new)):
#   data_array.append(data_new[i])
# # print(data_array)

# sentiment_pipeline = pipeline("sentiment-analysis")
# # data = ['love', 'bad']
# sentiment_pipeline(data_array)